In [1]:
# import all necessary packages & modules
import os
import time
import numpy as np
import pandas as pd
import json
import pyomo.environ as pyomo
from pyomo.opt import ProblemFormat
import matplotlib.pyplot as plt
import networkx as nx
import highspy
from networkx.readwrite import json_graph
from pyomo.opt import SolverStatus, TerminationCondition
from good_model_working import utils


import good_model_working
from good_model_working.reload import deep_reload
from good_model_working import opt_model
from good_model_working import diagnostics

#Change this to point to your version of cbc or use another solver
solver_name='appsi_highs'

In [2]:
# import the model data & sets
def load_data(filepath):        

# Load the pickled file
    with open(filepath, 'r') as file:
        # Load the JSON data into a Python dictionary
        data = json.load(file)
    return data

# Create a NetworkX graph from JSON data

model_sets = load_data('/Users/peterambiel/Desktop/good_model/Model Input/all_input_sets_sorted.json')
graph = json_graph.node_link_graph(load_data("/Users/peterambiel/Desktop/good_model/Model Input/graph.json"))

gen_to_remove = ['Fossil Waste', 
        'Municipal Solid Waste', 
        'Non-Fossil Waste', 
        'Pumped Storage',
        'Fuel Cell',
        'Landfill Gas', 
        "Energy Storage", 
        "Solar PV", 
        "Onshore Wind", 
        'New Battery Storage', 
        'IMPORT', 
        'Tires',
        'Offshore Wind', 
        'Solar Thermal'
        ]
        
sets = model_sets['gen_type']

model_sets['gen_type'] = [gen for gen in sets if not any(substring in gen for substring in gen_to_remove)]

In [3]:
# model testing
sub_nodes = [
    'ERC_REST', 'ERC_WEST', 'FRCC', 'MIS_AMSO', 'MIS_AR', 'MIS_D_MS', 'MIS_IA', 'MIS_IL', 
    'MIS_INKY', 'MIS_LA', 'MIS_LMI', 'MIS_MAPP', 'MIS_MIDA', 'MIS_MNWI', 'MIS_MO', 'MIS_WOTA', 
    'MIS_WUMS', 'NENGREST', 'NENG_CT', 'NENG_ME', 'NY_Z_A', 'NY_Z_B', 'NY_Z_C&E', 'NY_Z_D', 
    'NY_Z_F', 'NY_Z_G-I', 'NY_Z_J', 'NY_Z_K', 'PJM_AP', 'PJM_ATSI', 'PJM_COMD', 'PJM_Dom', 
    'PJM_EMAC', 'PJM_PENE', 'PJM_SMAC', 'PJM_WMAC', 'PJM_West', 'SPP_N', 'SPP_NEBR', 'SPP_SPS', 
    'SPP_WAUE', 'SPP_WEST', 'S_C_KY', 'S_C_TVA', 'S_D_AECI', 'S_SOU', 'S_VACA', 
    'ERC_FRNT', 'ERC_GWAY', 'SPP_KIAM', 'ERC_PHDL',
    'WECC_AZ', 'WECC_CO', 'WECC_ID', 'WECC_IID', 'WECC_MT', 'WECC_NM', 'WECC_NNV', 'WECC_PNW', 'WECC_SCE', 
    'WECC_SNV', 'WECC_UT', 'WECC_WY', 'WEC_BANC', 'WEC_CALN', 'WEC_LADW', 'WEC_SDGE', 
    ]

sub_nodes = sorted(sub_nodes)

subgraph = graph.subgraph(sub_nodes)

constraint_to_deactivate = ['generator', 'solar', 'wind', 'storage']

model_data = {
    'test_nodes': True,
    'test_cons': False,
    'graph': graph, 
    'sets': model_sets, 
    'subgraph_nodes': sub_nodes, 
    'subgraph': subgraph,
    'contraint_deactivation': None
}

In [ ]:
# import good_model_working
deep_reload(good_model_working)

# run model 
problem = good_model_working.opt_model.Opt_Model(model_data)

In [5]:
# print the objective value
print(f"Total system cost: ${pyomo.value(problem.model.obj_func):,.2f}")

Total system cost: $4,815,344,047.69


In [14]:
# get diagnostics
deep_reload(diagnostics)

hourly_mix = diagnostics.get_hourly_gen_mix(problem.results)
annual_mix = diagnostics.get_annual_gen_mix(problem.results)

In [ ]:
# plot diagnostics
deep_reload(diagnostics)

diagnostics.plot_hourly_gen_mix(hourly_mix)

In [ ]:
# plot diagnostics
deep_reload(diagnostics)

diagnostics.compare_annual_mix_to_baseline(annual_mix)